# Projet 8 : Traitez les images pour le système embarqué d’une voiture autonome

* [1. Mission](#partie1)
    * [1.1 Contexte](#partie1.1)
    * [1.2 Objectifs](#partie1.2)
    * [1.3 Enjeux](#partie1.3)
* [2. Préparation de l'environnement](#partie2)
    * [2.1 Installation des modules](#partie2.1)
    * [2.2 MLFlow](#partie2.2)
    * [2.3 Librairies](#partie2.3)
    * [2.4 Fonctions](#partie2.4)
* [3. Exploration des données (EDA)](#partie3)
    * [3.1 Analyse de la Structure des Dossiers et des Fichiers](#partie3.1)
    * [3.2 Visualisation des Images et des Masques](#partie3.2)
    * [3.3 Distribution des Classes dans les Masques](#partie3.3)
    * [3.4 Analyse des Dimensions des Images et des Masques](#partie3.4)
    * [3.5 Analyse des Formats des Fichiers](#partie3.5)
    * [3.6 Compléter le Filtrage des Groupes](#partie3.6)
    * [3.7 Vérification de la Qualité des Données](#partie3.7)
    * [3.8 Vérification de la Correspondance des Fichiers d'Annotations](#partie3.8)
* [4. Prétraitement des données](#partie4)
    * [4.1 Data augmentation](#partie4.1)
    * [4.2 Data generator](#partie4.2)
    * [4.3 Charger le pipeline d'augmentation et initialiser les générateurs](#partie4.3)
* [5. Modélisation](#partie5)
    * [5.1 Modèle U-Net Mini](#partie5.1)
    * [5.2 Modèle U-Net avec ResNet34](#partie5.2)
    * [5.3 Modèle U-Net avec EfficientNetB0](#partie5.3)
    * [5.4 Modèle SegNet avec VGG16](#partie5.4)
    * [5.5 Modèle DeepLabV3+ avec ResNet50](#partie5.5)
    * [5.6 Modèle PSPnet avec ResNet50](#partie5.6)
    * [5.7 Modèle FPN avec ResNet50](#partie5.7)
* [6. Conclusion](#partie6)

## <font color='red'>1. Mission</font><a class="anchor" id="partie1"></a>

##### <font color='blue'>1.1 Contexte</font><a class="anchor" id="partie1.1"></a>

Future Vision Transport est une entreprise innovante spécialisée dans la conception de systèmes embarqués de vision par ordinateur pour les véhicules autonomes. Ces systèmes visent à permettre aux véhicules d’analyser leur environnement en temps réel pour prendre des décisions précises et sécurisées.

En tant qu’ingénieur en intelligence artificielle au sein de l’équipe R&D, ma mission porte spécifiquement sur la segmentation d’images, une étape clé dans la chaîne du système embarqué.

Ce système se compose des quatre modules suivants :
- Acquisition des images en temps réel, qui capture les données visuelles.
- Traitement des images, qui prépare les images pour les étapes suivantes.
- Segmentation des images, votre domaine d’intervention.
- Système de décision, qui exploite les résultats de la segmentation pour guider le véhicule.

Notre travail s’intègre entre le module de traitement des images (2), géré par Franck, et le système de décision (4), dirigé par Laura. La collaboration avec ces deux parties prenantes a permis de définir les contraintes et attentes suivantes :

Franck, en charge du traitement des images, utilise le dataset Cityscapes, qui contient des annotations pour 32 sous-catégories. Cependant, seules les 8 catégories principales sont nécessaires pour notre tâche de segmentation.
Laura, en charge du système de décision, souhaite une API simple et efficace, capable de recevoir une image en entrée et de renvoyer un masque de segmentation en sortie.

##### <font color='blue'>1.2 Objectifs</font><a class="anchor" id="partie1.2"></a>

Notre mission consiste à développer un système performant et exploitable de segmentation d’images qui respecte les contraintes techniques et fonctionnelles suivantes :

- **Entraînement d’un modèle de segmentation sémantique** sur les 8 catégories principales du dataset Cityscapes, en utilisant Keras comme framework commun à toute l’équipe.
- **Création d’une API de prédiction** (via Flask ou FastAPI) pour permettre au système de décision de traiter facilement les résultats de la segmentation. Cette API devra être déployée sur le Cloud (Azure, Heroku, ou toute autre solution).
- **Conception d’une application web interactive** (via Flask ou Streamlit) pour tester l’API et visualiser les résultats des segmentations, en la rendant également accessible sur le Cloud.

##### <font color='blue'>1.3 Enjeux</font><a class="anchor" id="partie1.3"></a>

La segmentation d’images est essentielle pour les systèmes de conduite autonome, car elle permet de détecter et de catégoriser les éléments critiques de l’environnement, comme les routes, les véhicules et les piétons. Le modèle développé devra être à la fois performant et optimisé pour s’intégrer dans une chaîne de traitement en temps réel, tout en restant léger et rapide à déployer.

Ce projet représente une étape fondamentale dans le développement d’un système embarqué robuste et fiable pour les véhicules autonomes.

## <font color='red'>2. Préparation de l'environnement</font><a class="anchor" id="partie2"></a>

##### <font color='blue'>2.1 Installation des modules</font><a class="anchor" id="partie2.1"></a>

In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!pip install albumentations==1.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.0
    Uninstalling albumentations-2.0.0:
      Successfully uninstalled albumentations-2.0.0


In [ ]:
!pip install tensorflow==2.12 keras==2.12 segmentation-models==1.0.1

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of scikit-image to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 28.6 MB/s eta 0:00:00
   

In [ ]:
!pip install numpy==1.23.5

  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.0 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.31.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
pymc 5.19.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requi

##### <font color='blue'>2.2 MLFlow</font><a class="anchor" id="partie2.2"></a>

In [ ]:
import subprocess
from pyngrok import ngrok

# Démarrer MLFlow en arrière-plan
mlflow_server = subprocess.Popen(["mlflow", "ui", "--port", "5000"])

# Ajouter mon authtoken Ngrok (remplace par ton propre token)
!ngrok config add-authtoken 2o7fRSSTsKkRHY1jl4VoX9qS7AR_5TCYPvixQ8rv7g5PqJp8t

# Créer un tunnel pour accéder à MLFlow UI
public_url = ngrok.connect(5000)
print("MLFlow Tracking UI:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
MLFlow Tracking UI: NgrokTunnel: "https://8d8a-34-118-240-185.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import mlflow
mlflow.set_experiment("P8 - Image Segmentation")

2025/01/16 18:13:13 INFO mlflow.tracking.fluent: Experiment with name 'P8 - Image Segmentation' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/203819702506780593', creation_time=1737051193937, experiment_id='203819702506780593', last_update_time=1737051193937, lifecycle_stage='active', name='P8 - Image Segmentation', tags={}>

##### <font color='blue'>2.3 Librairies</font><a class="anchor" id="partie2.3"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from PIL import Image
import segmentation_models as sm
import albumentations as A
import tensorflow as tf
import time
import datetime
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Reshape, Lambda, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, jaccard_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU
import cv2
from segmentation_models.metrics import IOUScore, FScore
from segmentation_models.losses import DiceLoss
import json

##### <font color='blue'>2.4 Fonctions</font><a class="anchor" id="partie2.4"></a>

In [ ]:
def load_file_paths(images_path, masks_path, data_cat="train"):
    """
    Charge les chemins des images et des masques pour un type de données spécifique.

    Args:
        images_path (str): Chemin du dossier des images.
        masks_path (str): Chemin du dossier des masques.
        data_cat (str): Type de données ("train", "val", "test").

    Returns:
        list, list: Listes des chemins des images et des masques.
    """
    images_dir = os.path.join(images_path, data_cat)
    masks_dir = os.path.join(masks_path, data_cat)

    # Vérification des répertoires
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Le répertoire {images_dir} n'existe pas.")
    if not os.path.exists(masks_dir):
        raise FileNotFoundError(f"Le répertoire {masks_dir} n'existe pas.")

    cities = os.listdir(images_dir)
    image_files, mask_files = [], []

    for city in cities:
        city_img_dir = os.path.join(images_dir, city)
        city_mask_dir = os.path.join(masks_dir, city)

        # Vérifier si le sous-dossier existe dans "masks_path"
        if not os.path.exists(city_mask_dir):
            print(f"Attention : Le dossier des masques pour la ville '{city}' est manquant.")
            continue

        # Charger les fichiers
        city_images = [os.path.join(city_img_dir, f) for f in os.listdir(city_img_dir) if f.endswith("_leftImg8bit.png")]
        city_masks = [os.path.join(city_mask_dir, f) for f in os.listdir(city_mask_dir) if f.endswith("_gtFine_labelIds.png")]

        if len(city_images) != len(city_masks):
            print(f"Attention : Incohérence dans le nombre de fichiers pour la ville '{city}'.")

        image_files.extend(sorted(city_images))
        mask_files.extend(sorted(city_masks))

    return image_files, mask_files

In [ ]:
def show_image_and_mask(image_path, mask_path, save_path=None):
    """
    Affiche une image et son masque côte à côte.
    Si un chemin est fourni, enregistre la figure au format PNG.

    Args:
        image_path (str): Chemin de l'image RGB.
        mask_path (str): Chemin du masque associé.
        save_path (str, optional): Chemin pour sauvegarder l'image.
    """
    img = Image.open(image_path)
    mask = Image.open(mask_path)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(img)
    ax[0].set_title("Image RGB")
    ax[0].axis("off")

    ax[1].imshow(mask)
    ax[1].set_title("Masque (color)")
    ax[1].axis("off")

    # Enregistrer l’image si un chemin est fourni
    if save_path:
        fig.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
def analyze_class_distribution(masks_path):
    """
    Analyse la distribution des classes dans les masques d'annotations.

    Args:
        masks_path (str): Chemin vers le dossier contenant les masques.

    Returns:
        dict: Dictionnaire contenant la distribution des classes.
    """
    class_counts = Counter()

    for root, _, files in os.walk(masks_path):
        for file in files:
            if file.endswith("_gtFine_labelIds.png"):
                mask_path = os.path.join(root, file)
                mask = np.array(Image.open(mask_path))
                class_counts.update(mask.flatten())

    return class_counts

In [ ]:
def plot_class_distribution(class_counts, save_path=None):
    """
    Affiche et enregistre la distribution des classes sous forme de graphique.

    Args:
        class_counts (dict): Distribution des classes sous forme de dictionnaire.
        save_path (str, optional): Chemin pour sauvegarder le graphique. Si None, il affiche seulement.
    """
    plt.figure(figsize=(10, 5))
    plt.bar(class_counts.keys(), class_counts.values(), color="skyblue")
    plt.xlabel("Classes")
    plt.ylabel("Nombre de pixels")
    plt.title("Distribution des classes")

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

In [ ]:
def analyze_image_dimensions(images_path, masks_path):
    """
    Analyse les dimensions des images et des masques.

    Args:
        images_path (str): Chemin du dossier contenant les images.
        masks_path (str): Chemin du dossier contenant les masques.

    Returns:
        dict: Dictionnaire contenant les dimensions des images et des masques.
    """
    dimensions = {"images": [], "masks": []}

    for root, _, files in os.walk(images_path):
        for file in files:
            if file.endswith("_leftImg8bit.png"):
                img = Image.open(os.path.join(root, file))
                dimensions["images"].append(img.size)

    for root, _, files in os.walk(masks_path):
        for file in files:
            if file.endswith("_gtFine_labelIds.png"):
                mask = Image.open(os.path.join(root, file))
                dimensions["masks"].append(mask.size)

    # Compter la fréquence des dimensions
    img_dim_counts = Counter(dimensions["images"])
    mask_dim_counts = Counter(dimensions["masks"])

    return img_dim_counts, mask_dim_counts


In [ ]:
def analyze_file_formats(images_path, masks_path):
    image_formats = set()
    mask_formats = set()

    for root, _, files in os.walk(images_path):
        for file in files:
            image_formats.add(os.path.splitext(file)[1])

    for root, _, files in os.walk(masks_path):
        for file in files:
            mask_formats.add(os.path.splitext(file)[1])

    print("Formats des fichiers d'images :", image_formats)
    print("Formats des fichiers de masques :", mask_formats)

    return image_formats, mask_formats

In [ ]:
def filter_groups_in_mask(mask_path, class_to_group):
    mask = np.array(Image.open(mask_path))
    group_mask = np.zeros_like(mask)
    for cls, grp in class_to_group.items():
        group_mask[mask == cls] = grp
    return group_mask

def apply_cityscapes_palette(group_mask):
    cityscapes_palette = [
        (128, 64, 128),  # road (flat)
        (244, 35, 232),  # sidewalk (flat)
        (70, 70, 70),    # building (construction)
        (102, 102, 156), # wall (construction)
        (190, 153, 153), # fence (construction)
        (153, 153, 153), # pole (object)
        (250, 170, 30),  # traffic light (object)
        (220, 220, 0),   # traffic sign (object)
        (107, 142, 35),  # vegetation (nature)
        (152, 251, 152), # terrain (nature)
        (70, 130, 180),  # sky (sky)
        (220, 20, 60),   # person (human)
        (255, 0, 0),     # rider (human)
        (0, 0, 142),     # car (vehicle)
        (0, 0, 70),      # truck (vehicle)
        (0, 60, 100),    # bus (vehicle)
        (0, 80, 100),    # on rails (vehicle)
        (0, 0, 230),     # motorcycle (vehicle)
        (119, 11, 32),   # bicycle (vehicle)
        (0, 0, 0)        # void
    ] + [(0, 0, 0)] * (256 - 20)

    pil_mask = Image.fromarray(group_mask.astype('uint8'))
    flat_palette = [value for color in cityscapes_palette for value in color]
    pil_mask.putpalette(flat_palette)
    return pil_mask

In [ ]:
def check_file_integrity(images_path, masks_path):
    issues = {"unreadable_images": [], "unreadable_masks": []}

    # Vérifier les images
    for root, _, files in os.walk(images_path):
        for file in files:
            if file.endswith(".png"):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path)
                    img.verify()
                except Exception as e:
                    issues["unreadable_images"].append((file_path, str(e)))
                    os.remove(file_path)  # Suppression automatique

    # Vérifier les masques
    for root, _, files in os.walk(masks_path):
        for file in files:
            if file.endswith(".png"):
                file_path = os.path.join(root, file)
                try:
                    mask = Image.open(file_path)
                    mask.verify()
                except Exception as e:
                    issues["unreadable_masks"].append((file_path, str(e)))
                    os.remove(file_path)  # Suppression automatique

    print(f"Nombre d'images illisibles supprimées : {len(issues['unreadable_images'])}")
    print(f"Nombre de masques illisibles supprimés : {len(issues['unreadable_masks'])}")
    return issues

In [ ]:
def check_annotation_completeness(images_path, masks_path):
    """
    Vérifie que chaque image RGB a ses fichiers d'annotations associés.

    Args:
        images_path (str): Chemin vers le dossier contenant les images.
        masks_path (str): Chemin vers le dossier contenant les masques.

    Returns:
        list: Liste des problèmes de correspondance (image, fichier manquant).
    """
    issues = []
    for root, _, files in os.walk(images_path):
        for file in files:
            if file.endswith("_leftImg8bit.png"):
                base_name = file.replace("_leftImg8bit.png", "")
                mask_dir = root.replace("leftImg8bit", "gtFine")
                expected_files = [
                    f"{base_name}_gtFine_color.png",
                    f"{base_name}_gtFine_instanceIds.png",
                    f"{base_name}_gtFine_labelIds.png",
                    f"{base_name}_gtFine_polygons.json",
                ]
                # Vérifier la présence des fichiers
                for expected_file in expected_files:
                    if not os.path.exists(os.path.join(mask_dir, expected_file)):
                        issues.append((file, expected_file))
    return issues

## <font color='red'>3. Exploration des données (EDA)</font><a class="anchor" id="partie3"></a>

##### <font color='blue'>3.1 Analyse de la Structure des Dossiers et des Fichiers</font><a class="anchor" id="partie3.1"></a>

In [ ]:
# **Démarrer un run MLFlow**
with mlflow.start_run(run_name="EDA_Analyse_Structure"):

    # Définir les chemins vers les images et les masques
    images_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit"
    masks_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine"

    # Vérification des dossiers
    images_dirs = os.listdir(images_path)
    masks_dirs = os.listdir(masks_path)

    print("Contenu des dossiers :")
    print("Images :", images_dirs)
    print("Masques :", masks_dirs)

    # **Enregistrement des informations dans MLFlow**
    mlflow.log_param("Nombre de dossiers images", len(images_dirs))
    mlflow.log_param("Nombre de dossiers masques", len(masks_dirs))

    # **Chargement des chemins des fichiers (sans les intégrer à MLFlow maintenant)**
    train_images, train_masks = load_file_paths(images_path, masks_path, "train")
    val_images, val_masks = load_file_paths(images_path, masks_path, "val")
    test_images, test_masks = load_file_paths(images_path, masks_path, "test")

    # **Enregistrement des métriques dans MLFlow**
    mlflow.log_metric("nb_images_train", len(train_images))
    mlflow.log_metric("nb_masks_train", len(train_masks))
    mlflow.log_metric("nb_images_val", len(val_images))
    mlflow.log_metric("nb_masks_val", len(val_masks))
    mlflow.log_metric("nb_images_test", len(test_images))
    mlflow.log_metric("nb_masks_test", len(test_masks))

    print("Analyse de la structure des fichiers terminée et enregistrée dans MLFlow.")

##### <font color='blue'>3.2 Visualisation des Images et des Masques</font><a class="anchor" id="partie3.2"></a>

In [ ]:
import tempfile

with mlflow.start_run(run_name="EDA_Visualisation"):
    image_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png"
    mask_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/aachen_000000_000019_gtFine_color.png"

    # Créer un fichier temporaire pour sauvegarder l’image
    temp_img_path = tempfile.NamedTemporaryFile(suffix=".png", delete=False).name

    # Affichage et sauvegarde en même temps
    show_image_and_mask(image_path, mask_path, save_path=temp_img_path)

    # Enregistrement dans MLFlow
    mlflow.log_artifact(temp_img_path, artifact_path="EDA_Images")

    print(f"Image enregistrée dans MLFlow : {temp_img_path}")

##### <font color='blue'>3.3 Distribution des Classes dans les Masques</font><a class="anchor" id="partie3.3"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Distribution_Classes"):
    masks_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine/train"

    # Exécuter l'analyse
    class_counts = analyze_class_distribution(masks_path)

    # Loguer les métriques des classes
    for cls, count in class_counts.items():
        mlflow.log_metric(f"class_{cls}_pixels", count)

    # Sauvegarde du graphique dans MLFlow
    temp_plot_path = "/tmp/class_distribution.png"
    plot_class_distribution(class_counts, save_path=temp_plot_path)
    mlflow.log_artifact(temp_plot_path, artifact_path="EDA_Images")

##### <font color='blue'>3.4 Analyse des Dimensions des Images et des Masques</font><a class="anchor" id="partie3.4"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Analyse_Dimensions"):
    images_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit"
    masks_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine"

    # Exécuter l’analyse des dimensions
    img_dim_counts, mask_dim_counts = analyze_image_dimensions(images_path, masks_path)

    # Loguer les tailles uniques sous forme de paramètres
    mlflow.log_param("nb_dimensions_images_uniques", len(img_dim_counts))
    mlflow.log_param("nb_dimensions_masques_uniques", len(mask_dim_counts))

    # Loguer chaque dimension et sa fréquence
    for dim, count in img_dim_counts.items():
        dim_str = f"dim_img_{dim[0]}x{dim[1]}"
        mlflow.log_metric(dim_str, count)

    for dim, count in mask_dim_counts.items():
        dim_str = f"dim_mask_{dim[0]}x{dim[1]}"
        mlflow.log_metric(dim_str, count)


##### <font color='blue'>3.5 Analyse des Formats des Fichiers</font><a class="anchor" id="partie3.5"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Analyse_Formats"):
    images_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit"
    masks_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine"

    # Exécuter l’analyse des formats
    img_formats, mask_formats = analyze_file_formats(images_path, masks_path)

    # Loguer les formats uniques sous forme de paramètres
    mlflow.log_param("formats_images_uniques", ", ".join(sorted(img_formats)))
    mlflow.log_param("formats_masques_uniques", ", ".join(sorted(mask_formats)))

    # Loguer le nombre de fichiers par format sous forme de métriques
    for img_format in img_formats:
        img_format_clean = img_format.replace(".", "").lower()
        mlflow.log_metric(f"nb_fichiers_images_{img_format_clean}",
                          sum(1 for root, _, files in os.walk(images_path) if any(f.endswith(img_format) for f in files)))

    for mask_format in mask_formats:
        mask_format_clean = mask_format.replace(".", "").lower()
        mlflow.log_metric(f"nb_fichiers_masques_{mask_format_clean}",
                          sum(1 for root, _, files in os.walk(masks_path) if any(f.endswith(mask_format) for f in files)))


##### <font color='blue'>3.6 Compléter le Filtrage des Groupes</font><a class="anchor" id="partie3.6"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Filtrage_Groupes"):
    # Définition des classes et groupes
    class_to_group = {
        -1: 0, 0: 0, 1: 0, 2: 0,  # void
        7: 1, 8: 1, 9: 1, 10: 1,  # flat
        11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2,  # construction
        17: 3, 18: 3, 19: 3, 20: 3,  # object
        21: 4, 22: 4,  # nature
        23: 5,  # sky
        24: 6, 25: 6,  # human
        26: 7, 27: 7, 28: 7, 29: 7, 30: 7, 31: 7, 32: 7, 33: 7  # vehicle
    }

    example_mask_path = "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen/aachen_000000_000019_gtFine_labelIds.png"

    # Application du filtrage des groupes
    filtered_group_mask = filter_groups_in_mask(example_mask_path, class_to_group)
    colored_mask = apply_cityscapes_palette(filtered_group_mask)

    # Loguer le nombre de pixels par groupe
    unique, counts = np.unique(filtered_group_mask, return_counts=True)
    group_distribution = dict(zip(unique, counts))

    for group, count in group_distribution.items():
        mlflow.log_metric(f"group_{group}_pixels", count)

    # Sauvegarder l'image filtrée et colorisée dans MLFlow
    mask_output_path = "filtered_mask.png"
    colored_mask.save(mask_output_path)
    mlflow.log_artifact(mask_output_path)

    print("Filtrage des groupes et enregistrement des résultats terminés dans MLFlow.")

##### <font color='blue'>3.7 Vérification de la Qualité des Données</font><a class="anchor" id="partie3.7"></a>

In [ ]:
import json

with mlflow.start_run(run_name="EDA_Verification_Qualite"):
    # Vérification de l'intégrité des fichiers
    issues = check_file_integrity(
        "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit",
        "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine"
    )

    # Loguer les métriques
    mlflow.log_metric("images_illisibles_supprimees", len(issues["unreadable_images"]))
    mlflow.log_metric("masques_illisibles_supprimes", len(issues["unreadable_masks"]))

    # Sauvegarder la liste des fichiers supprimés dans MLFlow
    deleted_files_path = "deleted_files.json"
    with open(deleted_files_path, "w") as f:
        json.dump(issues, f, indent=4)

    mlflow.log_artifact(deleted_files_path)

    print("Vérification de la qualité des données terminée et enregistrée dans MLFlow.")


##### <font color='blue'>3.8 Vérification de la Correspondance des Fichiers d'Annotations</font><a class="anchor" id="partie3.8"></a>

In [ ]:
with mlflow.start_run(run_name="EDA_Verification_Annotations"):
    # Vérification de la correspondance des fichiers d'annotations
    annotation_issues = check_annotation_completeness(
        "/content/drive/My Drive/projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit",
        "/content/drive/My Drive/projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine",
    )

    # Loguer le nombre de fichiers manquants
    mlflow.log_metric("problèmes_annotations", len(annotation_issues))

    # Sauvegarder la liste des fichiers manquants dans MLFlow
    annotation_issues_path = "annotation_issues.json"
    with open(annotation_issues_path, "w") as f:
        json.dump(annotation_issues, f, indent=4)

    mlflow.log_artifact(annotation_issues_path)

    # Affichage des résultats dans le notebook
    if annotation_issues:
        print(f"Nombre de problèmes détectés : {len(annotation_issues)}")
        print("Exemples de problèmes :")
        for img, missing in annotation_issues[:10]:
            print(f"Image : {img}, Fichier manquant : {missing}")
    else:
        print("Toutes les images ont leurs fichiers d'annotations associés.")

    print("Vérification des annotations terminée et enregistrée dans MLFlow.")

## <font color='red'>4. Prétraitement des données (EDA)</font><a class="anchor" id="partie4"></a>

##### <font color='blue'>4.1 Data augmentation</font><a class="anchor" id="partie4.1"></a>

In [ ]:
def get_augmentation_pipeline():
    """
    Crée une pipeline d'augmentations dynamiques pour les images et les masques.
    Returns:
        albumentations.Compose: Pipeline d'augmentations.
    """
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Resize(height=256, width=256)
    ])

    # Loguer la configuration de l'augmentation dans MLFlow
    mlflow.log_param("augmentation_pipeline", str(augmentation))

    return augmentation

##### <font color='blue'>4.2 Data generator</font><a class="anchor" id="partie4.2"></a>

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    """
    Générateur de données avec augmentation dynamique pour l'entraînement et la validation.
    """

    def __init__(self, image_paths, mask_paths, batch_size, img_size=(256, 256), augmentation=None, shuffle=True, class_to_group=None, num_classes=8):
        """
        Initialisation du générateur de données.
        """
        assert len(image_paths) == len(mask_paths), "Les listes d'images et de masques doivent avoir la même longueur."
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.augmentation = augmentation
        self.shuffle = shuffle
        self.class_to_group = class_to_group
        self.num_classes = num_classes
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()

        # Loguer les paramètres du DataGenerator dans MLFlow
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("image_size", img_size)
        mlflow.log_param("num_classes", num_classes)
        mlflow.log_param("shuffle", shuffle)

    def __len__(self):
        """Nombre de lots par époque."""
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Génère un lot de données."""
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_paths_temp = [self.image_paths[k] for k in indexes]
        mask_paths_temp = [self.mask_paths[k] for k in indexes]
        X, y = self.__data_generation(image_paths_temp, mask_paths_temp)
        return X, y

    def on_epoch_end(self):
        """Mélange les données après chaque époque."""
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp, mask_paths_temp):
        """Prépare les lots."""
        X = np.empty((self.batch_size, *self.img_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, *self.img_size, self.num_classes), dtype=np.float32)

        i = 0
        while i < self.batch_size:
            idx = np.random.randint(len(image_paths_temp))
            image_path = image_paths_temp[idx]
            mask_path = mask_paths_temp[idx]

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.img_size[1], self.img_size[0]))

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.img_size[1], self.img_size[0]), interpolation=cv2.INTER_NEAREST)  # ✅ Correction

            unique_classes = np.unique(mask)
            if len(unique_classes) < 3:
                continue

            if self.augmentation:
                augmented = self.augmentation(image=image, mask=mask)
                image, mask = augmented['image'], augmented['mask']

            X[i] = image / 255.0
            y[i] = self._remap_and_one_hot_encode(mask)
            i += 1

        return X, y

    def _remap_and_one_hot_encode(self, mask):
        """Remappe les classes de masque et effectue un encodage one-hot."""
        remapped_mask = np.zeros_like(mask, dtype=np.int32)

        if self.class_to_group:
            for cls, grp in self.class_to_group.items():
                remapped_mask[mask == cls] = grp
        else:
            remapped_mask = mask

        one_hot_mask = tf.keras.utils.to_categorical(remapped_mask, num_classes=self.num_classes)
        return one_hot_mask.astype('float32')

    def compute_class_weights(self):
        """Calcule les poids pour chaque classe et les logue dans MLFlow."""
        pixel_counts = np.zeros(self.num_classes, dtype=np.int64)

        for mask_path in self.mask_paths:
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.img_size[::-1], interpolation=cv2.INTER_NEAREST)

            for cls, grp in self.class_to_group.items():
                pixel_counts[grp] += np.sum(mask == cls)

        total_pixels = np.sum(pixel_counts)
        class_weights = total_pixels / (self.num_classes * pixel_counts)
        normalized_weights = class_weights / np.sum(class_weights)

        # Loguer les poids des classes dans MLFlow
        for i, weight in enumerate(normalized_weights):
            mlflow.log_param(f"class_weight_{i}", float(weight))

        return normalized_weights

##### <font color='blue'>4.3 Charger le pipeline d'augmentation et initialiser les générateurs</font><a class="anchor" id="partie4.3"></a>

In [ ]:
with mlflow.start_run(run_name="Preprocessing"):
    # Charger les pipelines d'augmentation et les loguer dans MLFlow
    augmentation_pipeline = get_augmentation_pipeline()

    # Initialiser les générateurs
    train_gen = DataGenerator(
        train_images,
        train_masks,
        batch_size=16,
        img_size=(256, 256),
        augmentation=augmentation_pipeline,
        class_to_group=class_to_group,
        num_classes=8
    )

    val_gen = DataGenerator(
        val_images,
        val_masks,
        batch_size=16,
        img_size=(256, 256),
        class_to_group=class_to_group,
        num_classes=8
    )

    # Loguer la taille des images dans MLFlow
    mlflow.log_param("generator_img_size", train_gen.img_size)

    # Calculer et loguer les poids des classes
    class_weights = train_gen.compute_class_weights()
    mlflow.log_dict({"class_weights": class_weights.tolist()}, "class_weights.json")

## <font color='red'>5. Modélisation</font><a class="anchor" id="partie5"></a>

###### <font color='purple'>Fonctions de perte</font>

In [ ]:
from tensorflow.keras.losses import binary_crossentropy

def total_loss(y_true, y_pred):
    """
    Combine binary_crossentropy et dice_loss pour améliorer les performances globales.

    Args:
        y_true (Tensor): Masques réels.
        y_pred (Tensor): Masques prédits.

    Returns:
        Tensor: Valeur de la perte combinée.
    """
    loss = binary_crossentropy(y_true, y_pred) + (3 * dice_loss(y_true, y_pred))
    return loss


In [ ]:
def dice_coeff(y_true, y_pred):
    """
    Calcule le coefficient de Dice, une mesure de similarité pour la segmentation.

    Args:
        y_true (Tensor): Masques vrais (ground truth).
        y_pred (Tensor): Masques prédits.

    Returns:
        float: Coefficient de Dice.
    """
    smooth = 1.0
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

##### <font color='blue'>5.1 Modèle U-Net Mini</font><a class="anchor" id="partie5.1"></a>

Le modèle U-Net Mini sert de baseline dans cette étude. Il conserve la structure classique d'un U-Net (encodeur-décodeur avec connexions directes) mais avec un nombre réduit de paramètres, ce qui en fait une option légère et rapide à entraîner. Il est idéal pour évaluer les performances de base avant de tester des modèles plus complexes.



###### <font color='green'>Conception du Modèle</font>

In [ ]:
def unet_mini(input_size=(256, 256, 3), n_classes=32):
    """
    Implémente un modèle U-Net simplifié avec des convolutions dilatées.

    Args:
        input_size (tuple): Dimensions des images d'entrée (par défaut 256x256x3).
        n_classes (int): Nombre de classes pour la segmentation (par défaut 8).

    Returns:
        Model: Modèle U-Net Mini.
    """
    inputs = Input(input_size)

    # Encodeur
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Goulot d'étranglement avec convolution dilatée
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', dilation_rate=2)(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', dilation_rate=4)(c3)
    c3 = BatchNormalization()(c3)

    # Décodeur
    u1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(c3)
    u1 = concatenate([u1, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)

    u2 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(c4)
    u2 = concatenate([u2, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # Couche de sortie
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c5)

    return Model(inputs, outputs)

with mlflow.start_run(run_name="Unet Mini - Conception") as run:
    start_time = time.time()  # Début chrono

    # Instancier le modèle
    unet_model = unet_mini(input_size=(256, 256, 3), n_classes=8)

    # Afficher le résumé du modèle
    model_summary = []
    unet_model.summary(print_fn=lambda x: model_summary.append(x))
    model_summary = "\n".join(model_summary)

    # Enregistrer le résumé du modèle dans MLFlow
    mlflow.log_text(model_summary, "unet_mini_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("conception_time", elapsed_time)

print(f"Modèle U-Net Mini conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

Modèle U-Net Mini conçu et loggé dans MLFlow en 1.13 secondes.


###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
# Déclaration des métriques et de la fonction de perte
dice_loss = DiceLoss()
iou_score = IOUScore(threshold=0.5)
f_score = FScore(threshold=0.5)

with mlflow.start_run(run_name="Unet Mini - Compilation") as run:
    start_time = time.time()

    # Compilation du modèle
    unet_model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Log des hyperparamètres
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = unet_model.get_config()
    model_config_path = "unet_mini_config.json"
    with open(model_config_path, "w") as f:
        json.dump(model_config, f, indent=4)
    mlflow.log_artifact(model_config_path)

    # Temps d'exécution
    elapsed_time = time.time() - start_time
    mlflow.log_metric("compilation_time", elapsed_time)

print(f"Compilation terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")

Compilation terminée et loggée dans MLFlow en 0.24 secondes.


###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="Unet Mini - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Enregistrement des hyperparamètres
    mlflow.log_param("model_name", "Unet Mini")
    mlflow.log_param("epochs", 30)
    mlflow.log_param("batch_size", train_gen.batch_size)
    mlflow.log_param("image_size", train_gen.img_size)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("loss_function", "total_loss")
    mlflow.log_param("iou_score_metric", "IOUScore()")
    mlflow.log_param("f1_score_metric", "FScore(beta=1)")
    mlflow.log_param("dice_coeff_metric", "dice_coeff")

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_unet_mini"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/unet_mini_best_total_loss.h5",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle
    history = unet_model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque
    for epoch in range(len(history.history["loss"])):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/unet_mini_final_total_loss.h5"
    unet_model.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement
    history_serializable = {key: [float(v) for v in values] for key, values in history.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_unet_mini_total_loss"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

Epoch 1/30
185/185 [==============================] - 477s 3s/step - loss: 2.4380 - iou_score: 0.3397 - f1-score: 0.4375 - dice_coeff: 0.5533 - val_loss: 3.5900 - val_iou_score: 0.0490 - val_f1-score: 0.0716 - val_dice_coeff: 0.3723 - lr: 1.0000e-04
Epoch 2/30
185/185 [==============================] - 470s 3s/step - loss: 1.8349 - iou_score: 0.4850 - f1-score: 0.5855 - dice_coeff: 0.7512 - val_loss: 3.4705 - val_iou_score: 0.0852 - val_f1-score: 0.1289 - val_dice_coeff: 0.4133 - lr: 1.0000e-04
Epoch 3/30
185/185 [==============================] - 472s 3s/step - loss: 1.6578 - iou_score: 0.5292 - f1-score: 0.6392 - dice_coeff: 0.7866 - val_loss: 2.4816 - val_iou_score: 0.2814 - val_f1-score: 0.3836 - val_dice_coeff: 0.6412 - lr: 1.0000e-04
Epoch 4/30
185/185 [==============================] - 471s 3s/step - loss: 1.5290 - iou_score: 0.5550 - f1-score: 0.6693 - dice_coeff: 0.8059 - val_loss: 1.6518 - val_iou_score: 0.5085 - val_f1-score: 0.6297 - val_dice_coeff: 0.7737 - lr: 1.0000e-04


###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="Unet Mini - Evaluation") as run:
    start_time = time.time()

    # Extraire les métriques de l'entraînement
    history_dict = history.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("loss", []), label="Perte - Entraînement")
    ax.plot(epochs, history_dict.get("val_loss", []), label="Perte - Validation")
    ax.set_title("Courbe de la perte - U-Net Mini")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_unet_mini.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("iou_score", []), label="IoU - Entraînement")
    ax.plot(epochs, history_dict.get("val_iou_score", []), label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - U-Net Mini")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_unet_mini.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("dice_coeff", []), label="Dice Coefficient - Entraînement")
    ax.plot(epochs, history_dict.get("val_dice_coeff", []), label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - U-Net Mini")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_unet_mini.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = unet_model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - U-Net Mini")
    save_plot(fig, "confusion_matrix_unet_mini.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")

1/1 [==============================] - 1s 858ms/step
Évaluation terminée et sauvegardée dans MLFlow en 3.27 secondes.


In [ ]:
print("Métriques finales :")
print(f"Perte (Validation) : {history.history['val_loss'][-1]:.4f}")
print(f"IoU Score (Validation) : {history.history['val_iou_score'][-1]:.4f}")
print(f"F1 Score (Validation) : {history.history['val_f1-score'][-1]:.4f}")
print(f"Dice Coefficient (Validation) : {history.history['val_dice_coeff'][-1]:.4f}")

Métriques finales :
Perte (Validation) : 0.9778
IoU Score (Validation) : 0.6197
F1 Score (Validation) : 0.7360
Dice Coefficient (Validation) : 0.8516


##### <font color='blue'>5.2 Modèle U-Net avec ResNet34</font>

Ce modèle intègre ResNet34 comme backbone pour l’encodeur. Grâce à ses connexions résiduelles, ResNet34 améliore la stabilité de l'apprentissage tout en offrant un bon équilibre entre précision et efficacité. Le décodeur utilise les caractéristiques extraites pour reconstruire les masques segmentés.

###### <font color='green'>Conception du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="U-Net ResNet34 - Conception") as run:
    start_time = time.time()  # Démarrer le chronomètre

    # Définition du modèle U-Net avec ResNet34
    unet_resnet34 = sm.Unet(
        backbone_name="resnet34",
        input_shape=(256, 256, 3),
        classes=8,
        activation="softmax"
    )

    # Afficher et enregistrer le résumé du modèle
    model_summary = []
    unet_resnet34.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)

    # Enregistrer le résumé dans MLFlow directement
    mlflow.log_text(model_summary_str, "unet_resnet34_summary.txt")

    # Enregistrer les paramètres et le temps de conception
    mlflow.log_param("Backbone", "ResNet34")  # Enregistrer le backbone utilisé
    elapsed_time = time.time() - start_time  # Temps de conception
    mlflow.log_metric("conception_time", elapsed_time)

    print(f"Modèle U-Net ResNet34 conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
with mlflow.start_run(run_name="U-Net ResNet34 - Compilation"):
    start_time = time.time()  # Chrono

    # Fonction de perte et métriques
    dice_loss = sm.losses.DiceLoss()
    iou_score = sm.metrics.IOUScore(threshold=0.5)
    f_score = sm.metrics.FScore(threshold=0.5)

    # Compilation du modèle U-Net ResNet34
    unet_resnet34.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Log des hyperparamètres
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = unet_resnet34.get_config()
    model_config_path = "unet_resnet34_config.json"
    with open(model_config_path, "w") as f:
        json.dump(model_config, f, indent=4)
    mlflow.log_artifact(model_config_path)

    # Temps d'exécution
    elapsed_time = time.time() - start_time
    mlflow.log_metric("compilation_time", elapsed_time)

print(f"Compilation terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="Unet ResNet34 - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Enregistrement des hyperparamètres
    mlflow.log_param("epochs", 30)
    mlflow.log_param("batch_size", train_gen.batch_size)
    mlflow.log_param("image_size", train_gen.img_size)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("loss_function", "total_loss")
    mlflow.log_param("iou_score_metric", "IOUScore(threshold=0.5)")
    mlflow.log_param("f_score_metric", "FScore(threshold=0.5)")
    mlflow.log_param("dice_coeff_metric", "dice_coeff")

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_unet_resnet34"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/unet_resnet34_final_total_loss.h5",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle
    history_resnet34 = unet_resnet34.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque dans MLFlow
    for epoch in range(len(history_resnet34.history["loss"])):
        mlflow.log_metric("train_loss", history_resnet34.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_resnet34.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_resnet34.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_resnet34.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_resnet34.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_resnet34.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_resnet34.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_resnet34.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/unet_resnet34_final_total_loss.h5"
    unet_resnet34.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement en JSON
    history_serializable = {key: [float(v) for v in values] for key, values in history_resnet34.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_unet_resnet34_total_loss.json"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="U-Net ResNet34 - Evaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_resnet34.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("loss", []), label="Perte - Entraînement")
    ax.plot(epochs, history_dict.get("val_loss", []), label="Perte - Validation")
    ax.set_title("Courbe de la perte - U-Net ResNet34")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_unet_resnet34.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("iou_score", []), label="IoU - Entraînement")
    ax.plot(epochs, history_dict.get("val_iou_score", []), label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - U-Net ResNet34")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_unet_resnet34.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("dice_coeff", []), label="Dice Coefficient - Entraînement")
    ax.plot(epochs, history_dict.get("val_dice_coeff", []), label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - U-Net ResNet34")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_unet_resnet34.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = unet_resnet34.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - U-Net ResNet34")
    save_plot(fig, "confusion_matrix_unet_resnet34.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")

##### <font color='blue'>5.3 Modèle U-Net avec EfficientNetB0</font>

En utilisant EfficientNetB0 comme backbone, ce modèle mise sur une architecture optimisée pour l'efficacité et la précision. EfficientNetB0 utilise des techniques avancées de mise à l'échelle pour capturer des caractéristiques riches tout en limitant la complexité computationnelle.



###### <font color='green'>Conception du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="U-Net EfficientNetB0 - Conception") as run:
    start_time = time.time()  # Début du chrono

    # Définition du modèle U-Net avec EfficientNetB0 comme backbone
    unet_efficientnet = sm.Unet(
        backbone_name="efficientnetb0",
        input_shape=(256, 256, 3),
        classes=8,
        activation="softmax"
    )

    # Générer le résumé du modèle
    model_summary = []
    unet_efficientnet.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)

    # Enregistrement du résumé dans MLFlow
    summary_path = "unet_efficientnet_summary.txt"
    with open(summary_path, "w") as f:
        f.write(model_summary_str)

    mlflow.log_artifact(summary_path)  # Sauvegarde dans MLFlow
    mlflow.log_param("Backbone", "EfficientNetB0")  # Enregistrer le backbone utilisé

    # Mesurer le temps de conception
    elapsed_time = time.time() - start_time
    mlflow.log_metric("conception_time", elapsed_time)

    print(f"Modèle U-Net EfficientNetB0 conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
with mlflow.start_run(run_name="U-Net EfficientNetB0 - Compilation") as run:
    start_time = time.time()  # Début chrono

    # Déclaration des métriques et de la fonction de perte
    dice_loss = sm.losses.DiceLoss()
    iou_score = sm.metrics.IOUScore(threshold=0.5)
    f_score = sm.metrics.FScore(threshold=0.5)

    # Compilation du modèle
    unet_efficientnet.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Enregistrement des hyperparamètres
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = unet_efficientnet.get_config()
    config_path = "unet_efficientnet_config.json"
    with open(config_path, "w") as f:
        json.dump(model_config, f, indent=4)

    mlflow.log_artifact(config_path)

    # Temps de compilation
    elapsed_time = time.time() - start_time
    mlflow.log_metric("compilation_time", elapsed_time)

    print(f"Modèle U-Net EfficientNetB0 compilé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="U-Net EfficientNetB0 - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_unet_efficientnet"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/unet_efficientnet_best_total_loss.h5",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle
    history_efficientnet = unet_efficientnet.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque
    for epoch in range(len(history_efficientnet.history["loss"])):
        mlflow.log_metric("train_loss", history_efficientnet.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_efficientnet.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_efficientnet.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_efficientnet.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_efficientnet.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_efficientnet.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_efficientnet.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_efficientnet.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles//unet_efficientnet_final_total_loss.h5"
    unet_efficientnet.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement
    history_path = "/content/drive/My Drive/projet 8/Modeles//history_unet_efficientnet_total_loss.json"
    history_serializable = {key: [float(v) for v in values] for key, values in history_efficientnet.history.items()}
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

    print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="U-Net EfficientNetB0 - Évaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_efficientnet.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    # Fonction pour sauvegarder les courbes
    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("loss", []), label="Perte - Entraînement")
    ax.plot(epochs, history_dict.get("val_loss", []), label="Perte - Validation")
    ax.set_title("Courbe de la perte - U-Net EfficientNetB0")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_unet_efficientnet.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("iou_score", []), label="IoU - Entraînement")
    ax.plot(epochs, history_dict.get("val_iou_score", []), label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - U-Net EfficientNetB0")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_unet_efficientnet.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, history_dict.get("dice_coeff", []), label="Dice Coefficient - Entraînement")
    ax.plot(epochs, history_dict.get("val_dice_coeff", []), label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - U-Net EfficientNetB0")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_unet_efficientnet.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = unet_efficientnet.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - U-Net EfficientNetB0")
    save_plot(fig, "confusion_matrix_unet_efficientnet.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")


##### <font color='blue'>5.4 Modèle SegNet avec VGG16</font>

SegNet utilise VGG16 comme backbone et applique un décodeur basé sur des indices de pooling, récupérant ainsi des informations importantes des étapes d'encodage. Bien que simple et efficace, l’absence de connexions directes entre encodeur et décodeur peut limiter sa capacité à capturer des détails fins.

###### <font color='green'>Conception du Modèle</font>

In [ ]:
def segnet(input_size=(256, 256, 3), n_classes=8):
    """
    Implémente le modèle SegNet avec un encodeur basé sur VGG16 et un décodeur symétrique.

    Args:
        input_size (tuple): Dimensions des images en entrée (par défaut 256x256x3).
        n_classes (int): Nombre de classes de segmentation.

    Returns:
        Model: Modèle SegNet construit.
    """
    inputs = Input(input_size)

    # ENCODEUR (Basé sur VGG16)
    c1 = Conv2D(64, (3, 3), padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(64, (3, 3), padding='same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2, 2), strides=(2, 2))(c1)

    c2 = Conv2D(128, (3, 3), padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(128, (3, 3), padding='same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2))(c2)

    c3 = Conv2D(256, (3, 3), padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(256, (3, 3), padding='same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2, 2), strides=(2, 2))(c3)

    # DECODEUR
    u3 = UpSampling2D((2, 2))(p3)
    c4 = Conv2D(256, (3, 3), padding='same')(u3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(256, (3, 3), padding='same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    c5 = Conv2D(128, (3, 3), padding='same')(u2)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Conv2D(128, (3, 3), padding='same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    c6 = Conv2D(64, (3, 3), padding='same')(u1)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    c6 = Conv2D(64, (3, 3), padding='same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    # COUCHE DE SORTIE
    outputs = Conv2D(n_classes, (1, 1), activation="softmax")(c6)

    return Model(inputs, outputs)

# Démarrer un run MLFlow pour la conception du modèle
with mlflow.start_run(run_name="SegNet - Conception") as run:
    start_time = time.time()

    # Instancier le modèle
    segnet_model = segnet(input_size=(256, 256, 3), n_classes=8)

    # Afficher et enregistrer le résumé du modèle
    model_summary = []
    segnet_model.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)

    # Sauvegarde du résumé dans MLFlow
    mlflow.log_text(model_summary_str, "segnet_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("model_conception_time_seconds", elapsed_time)

print(f"Modèle SegNet conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
# Démarrer un run MLFlow pour la compilation du modèle
with mlflow.start_run(run_name="SegNet - Compilation") as run:
    start_time = time.time()

    # Définition des métriques et de la fonction de perte
    dice_loss = sm.losses.DiceLoss()
    iou_score = sm.metrics.IOUScore(threshold=0.5)
    f_score = sm.metrics.FScore(threshold=0.5)

    # Compilation du modèle SegNet
    segnet_model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Log des hyperparamètres
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = segnet_model.get_config()
    config_path = "segnet_config.json"
    with open(config_path, "w") as f:
        json.dump(model_config, f, indent=4)

    mlflow.log_artifact(config_path)

    # Temps de compilation
    elapsed_time = time.time() - start_time
    mlflow.log_metric("Compilation Time", elapsed_time)

print(f"Compilation du modèle SegNet terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
# Démarrer un run MLFlow pour l'entraînement du modèle SegNet
with mlflow.start_run(run_name="SegNet - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_segnet"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/segnet_best_total_loss.h5",
        save_best_only=True, monitor="val_loss", mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle SegNet
    history_segnet = segnet_model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque
    for epoch in range(len(history_segnet.history["loss"])):
        mlflow.log_metric("train_loss", history_segnet.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_segnet.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_segnet.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_segnet.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_segnet.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_segnet.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_segnet.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_segnet.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/segnet_final_total_loss.h5"
    segnet_model.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement
    history_serializable = {key: [float(v) for v in values] for key, values in history_segnet.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_segnet_total_loss.json"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

print(f"Entraînement du modèle SegNet terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
# Démarrer un run MLFlow pour l'évaluation du modèle SegNet
with mlflow.start_run(run_name="SegNet - Evaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_segnet.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    # Fonction pour sauvegarder et loguer une figure
    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, loss_train, label="Perte - Entraînement")
    ax.plot(epochs, loss_val, label="Perte - Validation")
    ax.set_title("Courbe de la perte - SegNet VGG16")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_unet_segnet.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, iou_train, label="IoU - Entraînement")
    ax.plot(epochs, iou_val, label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - SegNet VGG16")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_unet_segnet.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, dice_train, label="Dice Coefficient - Entraînement")
    ax.plot(epochs, dice_val, label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - SegNet VGG16")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_unet_segnet.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = segnet_model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - SegNet")
    save_plot(fig, "confusion_matrix_segnet.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time_seconds", eval_time)

    print(f"Évaluation du modèle SegNet terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")


##### <font color='blue'>5.5 Modèle DeepLabV3+ avec ResNet50</font>

Ce modèle exploite ResNet50 comme backbone pour extraire des caractéristiques profondes et intègre un module ASPP (Atrous Spatial Pyramid Pooling) pour capturer des contextes globaux à différentes échelles. Il est particulièrement performant pour les scènes complexes.

###### <font color='green'>Conception du Modèle</font>

In [ ]:
def deeplabv3plus(input_shape=(256, 256, 3), num_classes=8):
    """
    Implémente une version simplifiée de DeepLabV3+ avec ResNet50 comme backbone.

    Args:
        input_shape (tuple): Taille de l'image en entrée.
        num_classes (int): Nombre de classes pour la segmentation.

    Returns:
        keras.Model: Modèle DeepLabV3+.
    """
    # Charger ResNet50 comme Backbone (Sans la dernière couche)
    base_model = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_shape=input_shape
    )

    # Extraire les caractéristiques de la couche 'conv4_block6_out'
    x = base_model.get_layer("conv4_block6_out").output

    # ASPP (Atrous Spatial Pyramid Pooling)
    aspp = layers.Conv2D(256, (3, 3), dilation_rate=6, padding="same", activation="relu")(x)
    aspp = layers.BatchNormalization()(aspp)
    aspp = layers.Conv2D(256, (3, 3), dilation_rate=12, padding="same", activation="relu")(aspp)
    aspp = layers.BatchNormalization()(aspp)
    aspp = layers.Conv2D(256, (3, 3), dilation_rate=18, padding="same", activation="relu")(aspp)
    aspp = layers.BatchNormalization()(aspp)

    # Décodeur
    x = layers.Conv2DTranspose(256, (3, 3), strides=2, padding="same", activation="relu")(aspp)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D(size=(4, 4), interpolation="bilinear")(x)
    x = layers.Conv2D(num_classes, (1, 1), activation="softmax")(x)

    # Construire le modèle final
    model = keras.Model(inputs=base_model.input, outputs=x)
    return model

# Définition du modèle
with mlflow.start_run(run_name="DeepLabV3+ ResNet50 - Conception") as run:
    start_time = time.time()

    deeplab_model = deeplabv3plus(input_shape=(256, 256, 3), num_classes=8)

    # Afficher et sauvegarder le résumé du modèle
    model_summary = []
    deeplab_model.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)
    mlflow.log_text(model_summary_str, "deeplabv3plus_resnet50_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("model_conception_time_seconds", elapsed_time)

print(f"Modèle DeepLabV3+ ResNet50 conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

In [ ]:
print(deeplab_model)

###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
# Démarrer le tracking MLFlow pour la compilation du modèle
with mlflow.start_run(run_name="DeepLabV3+ - Compilation") as run:
    start_time = time.time()  # Début chrono

    # Déclaration des métriques et de la fonction de perte
    dice_loss = sm.losses.DiceLoss()
    iou_score = sm.metrics.IOUScore(threshold=0.5)
    f_score = sm.metrics.FScore(threshold=0.5)

    # Compilation du modèle
    deeplab_model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Enregistrer les paramètres de compilation dans MLFlow
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = deeplab_model.get_config()
    config_path = "deeplabv3plus_config.json"
    with open(config_path, "w") as f:
        json.dump(model_config, f, indent=4)

    mlflow.log_artifact(config_path)

    # Temps de compilation
    elapsed_time = time.time() - start_time
    mlflow.log_metric("Compilation Time", elapsed_time)

    print(f"Compilation terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
# Démarrer le tracking MLFlow pour l'entraînement du modèle
with mlflow.start_run(run_name="DeepLabV3+ - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_deeplab_resnet50"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)

    model_save_path = "/content/drive/My Drive/projet 8/Modeles/deeplab_resnet50_best_total_loss.h5"
    checkpoint_callback = ModelCheckpoint(
        model_save_path, save_best_only=True, monitor="val_loss", mode="min"
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Enregistrer les hyperparamètres de l'entraînement dans MLFlow
    mlflow.log_param("Epochs", 30)
    mlflow.log_param("Batch Size", train_gen.batch_size)
    mlflow.log_param("Image Size", train_gen.img_size)
    mlflow.log_param("Callbacks", ["EarlyStopping", "ModelCheckpoint", "ReduceLROnPlateau"])
    mlflow.log_param("EarlyStopping Patience", 3)
    mlflow.log_param("ReduceLR Factor", 0.5)
    mlflow.log_param("ReduceLR Patience", 3)
    mlflow.log_param("ReduceLR Min LR", 1e-6)

    # Entraînement du modèle
    history_deeplab = deeplab_model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques d’entraînement et validation par époque
    for epoch in range(len(history_deeplab.history["loss"])):
        mlflow.log_metric("train_loss", history_deeplab.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_deeplab.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_deeplab.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_deeplab.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_deeplab.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_deeplab.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_deeplab.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_deeplab.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/deeplab_resnet50_final_total_loss.h5"
    deeplab_model.save(final_model_path)
    mlflow.log_artifact(final_model_path)

  # Sauvegarde de l'historique d'entraînement
    history_serializable = {key: [float(v) for v in values] for key, values in history_deeplab.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_deeplab_resnet50_total_loss.json"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

    print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="DeepLabV3+ ResNet50 - Évaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_deeplab.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    # Fonction pour sauvegarder les courbes
    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, loss_train, label="Perte - Entraînement")
    ax.plot(epochs, loss_val, label="Perte - Validation")
    ax.set_title("Courbe de la perte - DeepLabV3+ ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_deeplabv3+plus.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, iou_train, label="IoU - Entraînement")
    ax.plot(epochs, iou_val, label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - DeepLabV3+ ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_deeplabv3+plus.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, dice_train, label="Dice Coefficient - Entraînement")
    ax.plot(epochs, dice_val, label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - DeepLabV3+ ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_deeplabv3+plus.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = deeplab_model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - DeepLabV3+ ResNet50")
    save_plot(fig, "confusion_matrix_deeplabv3+plus.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")


##### <font color='blue'>5.6 Modèle PSPnet</font>

Le modèle PSPNet utilise également ResNet50 comme backbone, mais se distingue par son pooling pyramidal, qui combine des informations locales et globales. Cette approche est idéale pour segmenter des objets de tailles variées dans une scène.

###### <font color='green'>Conception du Modèle</font>

In [ ]:
def pspnet(input_shape=(288, 288, 3), num_classes=8):  # Changement de 256x256 → 288x288
    """
    Implémente PSPNet avec un backbone ResNet50.

    Args:
        input_shape (tuple): Taille de l'image en entrée.
        num_classes (int): Nombre de classes pour la segmentation.

    Returns:
        keras.Model: Modèle PSPNet.
    """
    model = sm.PSPNet(
        backbone_name="resnet50",
        input_shape=input_shape,
        classes=num_classes,
        activation="softmax"
    )
    return model

# Définition du modèle
with mlflow.start_run(run_name="PSPNet ResNet50 - Conception") as run:
    start_time = time.time()

    pspnet_model = pspnet(input_shape=(288, 288, 3), num_classes=8)  # Taille corrigée

    # Afficher et sauvegarder le résumé du modèle
    model_summary = []
    pspnet_model.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)
    mlflow.log_text(model_summary_str, "pspnet_resnet50_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("model_conception_time_seconds", elapsed_time)

print(f"Modèle PSPNet ResNet50 conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
# Démarrer le tracking MLFlow pour la compilation du modèle
with mlflow.start_run(run_name="PSPNet ResNet50 - Compilation") as run:
    start_time = time.time()

    # Déclaration des métriques et de la fonction de perte
    dice_loss = DiceLoss()
    iou_score = IOUScore(threshold=0.5)
    f_score = FScore(threshold=0.5)

    # Compilation du modèle
    pspnet_model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Enregistrer les paramètres de compilation dans MLFlow
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = pspnet_model.get_config()
    config_path = "pspnet_config.json"
    with open(config_path, "w") as f:
        json.dump(model_config, f, indent=4)

    mlflow.log_artifact(config_path)

    # Temps de compilation
    elapsed_time = time.time() - start_time
    mlflow.log_metric("Compilation Time", elapsed_time)

    print(f"Compilation terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="PSPNet ResNet50 - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Enregistrement des hyperparamètres
    mlflow.log_param("model_name", "PSPNet ResNet50")
    mlflow.log_param("epochs", 30)
    mlflow.log_param("batch_size", train_gen.batch_size)
    mlflow.log_param("image_size", train_gen.img_size)

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_pspnet"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/pspnet_best.h5_total_loss",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle
    history_pspnet = pspnet_model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque
    for epoch in range(len(history_pspnet.history["loss"])):
        mlflow.log_metric("train_loss", history_pspnet.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_pspnet.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_pspnet.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_pspnet.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_pspnet.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_pspnet.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_pspnet.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_pspnet.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/pspnet_final_total_loss.h5"
    pspnet_model.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement
    history_serializable = {key: [float(v) for v in values] for key, values in history_pspnet.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_pspnet_total_loss.json"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="PSPNet - Évaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_pspnet.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    # Fonction pour sauvegarder les courbes
    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, loss_train, label="Perte - Entraînement")
    ax.plot(epochs, loss_val, label="Perte - Validation")
    ax.set_title("Courbe de la perte - PSPNet")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_pspnet.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, iou_train, label="IoU - Entraînement")
    ax.plot(epochs, iou_val, label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - PSPNet")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_pspnet.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, dice_train, label="Dice Coefficient - Entraînement")
    ax.plot(epochs, dice_val, label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - PSPNet")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_pspnet.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = pspnet_model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - PSPNet")
    save_plot(fig, "confusion_matrix_pspnet.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")

##### <font color='blue'>5.7 Modèle FPN</font>

Le modèle FPN (Feature Pyramid Network) avec ResNet50 combine les caractéristiques des couches profondes et superficielles du backbone pour obtenir une segmentation équilibrée entre détails fins et contexte global.

###### <font color='green'>Conception du Modèle</font>

In [ ]:
def fpn_model(input_shape=(512, 512, 3), num_classes=8):
    """
    Implémente le modèle Feature Pyramid Network (FPN) avec ResNet50 comme backbone.

    Args:
        input_shape (tuple): Taille de l'image en entrée.
        num_classes (int): Nombre de classes pour la segmentation.

    Returns:
        keras.Model: Modèle FPN.
    """
    model = sm.FPN(
        backbone_name="resnet50",
        input_shape=input_shape,
        classes=num_classes,
        activation="softmax"
    )
    return model

# Définition du modèle
with mlflow.start_run(run_name="FPN ResNet50 - Conception") as run:
    start_time = time.time()

    fpn_resnet50 = fpn_model(input_shape=(512, 512, 3), num_classes=8)

    # Afficher et sauvegarder le résumé du modèle
    model_summary = []
    fpn_resnet50.summary(print_fn=lambda x: model_summary.append(x))
    model_summary_str = "\n".join(model_summary)
    mlflow.log_text(model_summary_str, "fpn_resnet50_summary.txt")

    # Temps de conception du modèle
    elapsed_time = time.time() - start_time
    mlflow.log_metric("conception_time", elapsed_time)

print(f"Modèle FPN ResNet50 conçu et loggé dans MLFlow en {elapsed_time:.2f} secondes.")


###### <font color='green'>Compilateur et Fonctions de Perte</font>

In [ ]:
# Démarrer le tracking MLFlow pour la compilation du modèle
with mlflow.start_run(run_name="FPN ResNet50 - Compilation") as run:
    start_time = time.time()  # Début chrono

    # Déclaration des métriques et de la fonction de perte
    dice_loss = DiceLoss()
    iou_score = IOUScore(threshold=0.5)
    f_score = FScore(threshold=0.5)

    # Compilation du modèle
    fpn_resnet50.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=total_loss,
        metrics=[iou_score, f_score, dice_coeff]
    )

    # Enregistrer les paramètres de compilation dans MLFlow
    mlflow.log_param("Optimizer", "Adam")
    mlflow.log_param("Learning Rate", 1e-4)
    mlflow.log_param("Loss Function", "total_loss")
    mlflow.log_param("IoU Threshold", 0.5)
    mlflow.log_param("FScore Threshold", 0.5)
    mlflow.log_param("Dice Coefficient Metric", "dice_coeff")

    # Sauvegarde de la configuration du modèle
    model_config = fpn_resnet50.get_config()
    config_path = "fpn_resnet50_config.json"
    with open(config_path, "w") as f:
        json.dump(model_config, f, indent=4)

    mlflow.log_artifact(config_path)

    # Temps de compilation
    elapsed_time = time.time() - start_time
    mlflow.log_metric("Compilation Time", elapsed_time)

    print(f"Compilation terminée et loggée dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Entraînement du Modèle</font>

In [ ]:
with mlflow.start_run(run_name="FPN ResNet50 - Entraînement") as run:
    start_time = time.time()  # Début chrono

    # Enregistrement des hyperparamètres
    mlflow.log_param("model_name", "FPN ResNet50")
    mlflow.log_param("epochs", 30)
    mlflow.log_param("batch_size", train_gen.batch_size)
    mlflow.log_param("image_size", train_gen.img_size)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("loss_function", "total_loss")
    mlflow.log_param("iou_score_metric", "IOUScore()")
    mlflow.log_param("f1_score_metric", "FScore(beta=1)")
    mlflow.log_param("dice_coeff_metric", "dice_coeff")

    # Préparer les callbacks
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_fpn_resnet50"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
    checkpoint_callback = ModelCheckpoint(
        "/content/drive/My Drive/projet 8/Modeles/fpn_resnet50_best_total_loss.h5",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    )
    early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    callbacks = [tensorboard_callback, checkpoint_callback, early_stopping, reduce_lr]

    # Entraînement du modèle
    history_fpn = fpn_resnet50.fit(
        train_gen,
        validation_data=val_gen,
        epochs=30,
        callbacks=callbacks
    )

    # Enregistrement des métriques par époque
    for epoch in range(len(history_fpn.history["loss"])):
        mlflow.log_metric("train_loss", history_fpn.history["loss"][epoch], step=epoch)
        mlflow.log_metric("val_loss", history_fpn.history["val_loss"][epoch], step=epoch)
        mlflow.log_metric("train_iou", history_fpn.history["iou_score"][epoch], step=epoch)
        mlflow.log_metric("val_iou", history_fpn.history["val_iou_score"][epoch], step=epoch)
        mlflow.log_metric("train_f1", history_fpn.history["f1-score"][epoch], step=epoch)
        mlflow.log_metric("val_f1", history_fpn.history["val_f1-score"][epoch], step=epoch)
        mlflow.log_metric("train_dice_coeff", history_fpn.history["dice_coeff"][epoch], step=epoch)
        mlflow.log_metric("val_dice_coeff", history_fpn.history["val_dice_coeff"][epoch], step=epoch)

    # Sauvegarde du modèle final
    final_model_path = "/content/drive/My Drive/projet 8/Modeles/fpn_resnet50_final_total_loss.h5"
    fpn_resnet50.save(final_model_path)
    mlflow.log_artifact(final_model_path)

    # Sauvegarde de l'historique d'entraînement
    history_serializable = {key: [float(v) for v in values] for key, values in history_fpn.history.items()}
    history_path = "/content/drive/My Drive/projet 8/Modeles/history_fpn_resnet50_total_loss.json"
    with open(history_path, "w") as f:
        json.dump(history_serializable, f, indent=4)

    mlflow.log_artifact(history_path)

    # Temps d'exécution total
    elapsed_time = time.time() - start_time
    mlflow.log_metric("training_time", elapsed_time)

    print(f"Entraînement terminé et loggé dans MLFlow en {elapsed_time:.2f} secondes.")

###### <font color='green'>Évaluation et Visualisation</font>

In [ ]:
with mlflow.start_run(run_name="FPN ResNet50 - Évaluation") as run:
    start_time = time.time()

    # Extraire les métriques d'entraînement
    history_dict = history_fpn.history
    epochs = range(1, len(history_dict.get("loss", [])) + 1)

    # Vérification et récupération des métriques
    loss_train = history_dict.get("loss", [])
    loss_val = history_dict.get("val_loss", [])
    iou_train = history_dict.get("iou_score", [])
    iou_val = history_dict.get("val_iou_score", [])
    f1_train = history_dict.get("f1-score", [])
    f1_val = history_dict.get("val_f1-score", [])
    dice_train = history_dict.get("dice_coeff", [])
    dice_val = history_dict.get("val_dice_coeff", [])

    # Fonction pour sauvegarder les courbes
    def save_plot(fig, filename):
        """
        Sauvegarde une figure Matplotlib et l'enregistre dans MLFlow.

        Args:
            fig (matplotlib.figure.Figure): La figure Matplotlib à sauvegarder.
            filename (str): Nom du fichier sous lequel l'image sera sauvegardée.
        """
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close(fig)
        os.remove(filename)  # Nettoyage après logging

    # Courbe de la perte
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, loss_train, label="Perte - Entraînement")
    ax.plot(epochs, loss_val, label="Perte - Validation")
    ax.set_title("Courbe de la perte - FPN ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Perte")
    ax.legend()
    save_plot(fig, "loss_curve_fpn_resnet50.png")

    # Courbe de l'IoU
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, iou_train, label="IoU - Entraînement")
    ax.plot(epochs, iou_val, label="IoU - Validation")
    ax.set_title("Courbe de l'IoU - FPN ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("IoU")
    ax.legend()
    save_plot(fig, "iou_curve_fpn_resnet50.png")

    # Courbe du Dice Coefficient
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(epochs, dice_train, label="Dice Coefficient - Entraînement")
    ax.plot(epochs, dice_val, label="Dice Coefficient - Validation")
    ax.set_title("Courbe du Dice Coefficient - FPN ResNet50")
    ax.set_xlabel("Époques")
    ax.set_ylabel("Dice Coefficient")
    ax.legend()
    save_plot(fig, "dice_curve_fpn_resnet50.png")

    # Matrice de confusion
    X_val, y_val = val_gen[0]
    y_pred = fpn_resnet50.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=-1)
    y_true_classes = np.argmax(y_val, axis=-1)

    y_pred_flat = y_pred_classes.flatten()
    y_true_flat = y_true_classes.flatten()

    conf_matrix = confusion_matrix(y_true_flat, y_pred_flat, labels=range(8))
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=['Void', 'Flat', 'Construction', 'Object', 'Nature', 'Sky', 'Human', 'Vehicle'])

    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax, cmap="viridis", values_format="d")
    plt.xticks(rotation=45)
    plt.title("Matrice de confusion des groupes - FPN ResNet50")
    save_plot(fig, "confusion_matrix_fpn_resnet50.png")

    # Enregistrement des scores finaux dans MLFlow
    final_metrics = {
        "Final Loss": loss_val[-1] if loss_val else None,
        "Final IoU Score": iou_val[-1] if iou_val else None,
        "Final F1-Score": f1_val[-1] if f1_val else None,
        "Final Dice Coefficient": dice_val[-1] if dice_val else None,
    }

    for metric, value in final_metrics.items():
        if value is not None:
            mlflow.log_metric(metric, value)

    # Temps d'évaluation total
    eval_time = time.time() - start_time
    mlflow.log_metric("evaluation_time", eval_time)

    print(f"Évaluation terminée et sauvegardée dans MLFlow en {eval_time:.2f} secondes.")

## <font color='red'>6. Conclusion</font><a class="anchor" id="partie6"></a>

Après avoir testé plusieurs modèles de segmentation d’images avec différents backbones,
le modèle **FPN avec ResNet50** s'est révélé être le plus performant.

Résultats clés :
- **IoU Score** : 0.770, le plus élevé parmi tous les modèles testés.
- **Dice Score** : 0.917, confirmant une segmentation précise.
- **Loss** : 0.506, indiquant une bonne convergence pendant l'entraînement.
- Temps d'entraînement raisonnable, équilibrant performance et complexité.

Comparaison avec les autres modèles :
- **U-Net Mini**, utilisé comme baseline, a montré des performances modestes (IoU : 0.626).
- Les architectures avancées comme **DeepLabV3+** et **PSPNet** ont offert des résultats solides, mais elles n'ont pas surpassé le FPN avec ResNet50.

Choix final :
- Le **FPN avec ResNet50** a été retenu comme modèle optimal pour sa capacité à segmenter les scènes complexes tout en préservant les détails critiques.
- Il sera intégré dans l'API de prédiction pour fournir des masques de segmentation en temps réel.

Perspectives :
- Bien que le modèle soit performant, des pistes d'amélioration existent :
- Entraîner le modèle avec des images en haute résolution pour plus de précision.
- Explorer des backbones plus avancés, comme EfficientNetB7, pour des scènes complexes.